## FashionMNIST

In [1]:
!pip install torch
!pip install torchvision cleverhans

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for nvidia-curand-cu12==10.3.5.147 from https://files.pythonhosted.org/packages/8a/6d/44ad094874c6f1b9c654f8ed939590bdc408349f137f9b98a3a23ccec411/nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Obtaining dependency information for nvidia-nvjitlink-cu12==12.4.127 from https://files.pythonhosted.org/packages/ff/ff/847841bacfbefc97a00036e0fce5a0f086b640756dc38caea5e1bb002655/nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl (56.3 MB)
Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (21.1 MB)
Defaulting to user installation because 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Load FashionMNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform),
    batch_size=1000, shuffle=False
)

In [3]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(weights=None)  # Use weights=None instead of pretrained=False
        # Modify the first conv layer to accept 1-channel input
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # Remove maxpool layer to reduce spatial downsampling
        self.model.maxpool = nn.Identity()
        # Modify the fully connected layer to output 10 classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [4]:
# FGSM attack implementation
def fgsm_attack(model, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = F.cross_entropy(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbed_data = data + epsilon * data_grad.sign()
    return perturbed_data


In [5]:
# Adversarial training with accuracy metrics
def train(model, device, train_loader, optimizer, epsilon, epoch):
    model.train()
    total_loss, total_adv_loss = 0, 0
    correct, correct_adv = 0, 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Standard forward pass
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        total_loss += loss.item()

        # Compute accuracy for clean data
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        # Generate adversarial examples using FGSM
        adv_data = fgsm_attack(model, data, target, epsilon)
        adv_output = model(adv_data)
        adv_loss = F.cross_entropy(adv_output, target)
        total_adv_loss += adv_loss.item()

        # Compute accuracy for adversarial data
        adv_pred = adv_output.argmax(dim=1, keepdim=True)
        correct_adv += adv_pred.eq(target.view_as(adv_pred)).sum().item()

        # Combined loss
        total_batch_loss = (loss + adv_loss) / 2

        # Backward pass and optimization
        total_batch_loss.backward()
        optimizer.step()

        total += target.size(0)

        # Log information for each batch
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] '
                  f'Loss: {loss.item():.4f} | Adv Loss: {adv_loss.item():.4f} | '
                  f'Acc: {100. * correct / total:.2f}% | Adv Acc: {100. * correct_adv / total:.2f}%')

    # Log average loss and accuracy for epoch
    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    avg_acc = 100. * correct / total
    avg_adv_acc = 100. * correct_adv / total
    print(f'==> Epoch: {epoch+1} | Avg Loss: {avg_loss:.4f} | Avg Adv Loss: {avg_adv_loss:.4f} | '
          f'Avg Acc: {avg_acc:.2f}% | Avg Adv Acc: {avg_adv_acc:.2f}%')


In [6]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model = ResNet18().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Using device: {device}")

Using device: cuda


In [7]:
print(device)

cuda


In [8]:
# Training loop
epochs = 5
epsilon = 0.1  
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epsilon, epoch)
    print(f"Epoch {epoch+1}/{epochs} completed\n")

print("Adversarial training completed.")

Epoch: 1 [0/60000] Loss: 2.4609 | Adv Loss: 2.9688 | Acc: 9.38% | Adv Acc: 1.56%
Epoch: 1 [6400/60000] Loss: 0.7250 | Adv Loss: 1.0568 | Acc: 74.26% | Adv Acc: 58.23%
Epoch: 1 [12800/60000] Loss: 0.4496 | Adv Loss: 0.7671 | Acc: 78.56% | Adv Acc: 63.99%
Epoch: 1 [19200/60000] Loss: 0.2926 | Adv Loss: 0.5025 | Acc: 80.69% | Adv Acc: 66.81%
Epoch: 1 [25600/60000] Loss: 0.4391 | Adv Loss: 0.6936 | Acc: 81.82% | Adv Acc: 68.45%
Epoch: 1 [32000/60000] Loss: 0.3118 | Adv Loss: 0.5052 | Acc: 82.82% | Adv Acc: 69.76%
Epoch: 1 [38400/60000] Loss: 0.2165 | Adv Loss: 0.4870 | Acc: 83.52% | Adv Acc: 70.80%
Epoch: 1 [44800/60000] Loss: 0.3683 | Adv Loss: 0.6605 | Acc: 84.11% | Adv Acc: 71.81%
Epoch: 1 [51200/60000] Loss: 0.4115 | Adv Loss: 0.9137 | Acc: 84.59% | Adv Acc: 72.40%
Epoch: 1 [57600/60000] Loss: 0.4035 | Adv Loss: 0.7646 | Acc: 84.88% | Adv Acc: 72.90%
==> Epoch: 1 | Avg Loss: 0.3994 | Avg Adv Loss: 0.6955 | Avg Acc: 85.00% | Avg Adv Acc: 73.07%
Epoch 1/5 completed

Epoch: 2 [0/60000] Lo

Sp, The decrease in both average losses over epochs suggests the model is improving on both clean and adversarial examples, learning to classify both correctly.

## ResNet-50

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Load FashionMNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform),
    batch_size=1000, shuffle=False
)


In [10]:
# Define a ResNet50 model adapted for grayscale images and 10 classes
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.model = models.resnet50(weights=None)  # Use weights=None instead of pretrained=False
        # Modify the first conv layer to accept 1-channel input
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # Remove maxpool layer to reduce spatial downsampling
        self.model.maxpool = nn.Identity()
        # Modify the fully connected layer to output 10 classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [11]:
# FGSM attack implementation
def fgsm_attack(model, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = F.cross_entropy(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbed_data = data + epsilon * data_grad.sign()
    return perturbed_data

In [12]:
# Adversarial training with accuracy metrics
def train(model, device, train_loader, optimizer, epsilon, epoch):
    model.train()
    total_loss, total_adv_loss = 0, 0
    correct, correct_adv = 0, 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Standard forward pass
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        total_loss += loss.item()

        # Compute accuracy for clean data
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        # Generate adversarial examples using FGSM
        adv_data = fgsm_attack(model, data, target, epsilon)
        adv_output = model(adv_data)
        adv_loss = F.cross_entropy(adv_output, target)
        total_adv_loss += adv_loss.item()

        # Compute accuracy for adversarial data
        adv_pred = adv_output.argmax(dim=1, keepdim=True)
        correct_adv += adv_pred.eq(target.view_as(adv_pred)).sum().item()

        # Combined loss
        total_batch_loss = (loss + adv_loss) / 2

        # Backward pass and optimization
        total_batch_loss.backward()
        optimizer.step()

        total += target.size(0)

        # Log information for each batch
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] '
                  f'Loss: {loss.item():.4f} | Adv Loss: {adv_loss.item():.4f} | '
                  f'Acc: {100. * correct / total:.2f}% | Adv Acc: {100. * correct_adv / total:.2f}%')

    # Log average loss and accuracy for epoch
    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    avg_acc = 100. * correct / total
    avg_adv_acc = 100. * correct_adv / total
    print(f'==> Epoch: {epoch+1} | Avg Loss: {avg_loss:.4f} | Avg Adv Loss: {avg_adv_loss:.4f} | '
          f'Avg Acc: {avg_acc:.2f}% | Avg Adv Acc: {avg_adv_acc:.2f}%')


In [13]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Using device: {device}")

Using device: cuda


In [14]:
# Training loop
epochs = 5
epsilon = 0.1  
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epsilon, epoch)
    print(f"Epoch {epoch+1}/{epochs} completed\n")

print("Adversarial training completed.")

Epoch: 1 [0/60000] Loss: 2.4655 | Adv Loss: 2.4578 | Acc: 10.94% | Adv Acc: 12.50%
Epoch: 1 [6400/60000] Loss: 0.7652 | Adv Loss: 1.1625 | Acc: 67.76% | Adv Acc: 55.00%
Epoch: 1 [12800/60000] Loss: 0.6373 | Adv Loss: 1.0035 | Acc: 74.68% | Adv Acc: 62.06%
Epoch: 1 [19200/60000] Loss: 0.3459 | Adv Loss: 0.5804 | Acc: 77.71% | Adv Acc: 65.14%
Epoch: 1 [25600/60000] Loss: 0.4209 | Adv Loss: 0.7779 | Acc: 79.58% | Adv Acc: 67.18%
Epoch: 1 [32000/60000] Loss: 0.4691 | Adv Loss: 0.8537 | Acc: 80.68% | Adv Acc: 68.46%
Epoch: 1 [38400/60000] Loss: 0.2549 | Adv Loss: 0.4828 | Acc: 81.68% | Adv Acc: 69.66%
Epoch: 1 [44800/60000] Loss: 0.3454 | Adv Loss: 0.6307 | Acc: 82.31% | Adv Acc: 70.39%
Epoch: 1 [51200/60000] Loss: 0.2989 | Adv Loss: 0.5535 | Acc: 82.90% | Adv Acc: 71.20%
Epoch: 1 [57600/60000] Loss: 0.4240 | Adv Loss: 0.7741 | Acc: 83.47% | Adv Acc: 71.74%
==> Epoch: 1 | Avg Loss: 0.4470 | Avg Adv Loss: 0.7546 | Avg Acc: 83.61% | Avg Adv Acc: 71.88%
Epoch 1/5 completed

Epoch: 2 [0/60000] 

Avg Loss:

This is the average loss on clean (non-adversarial) images across the entire epoch. Lower values indicate that the model’s predictions are closer to the true labels.
Over the epochs, Avg Loss decreases significantly (from 0.4470 in Epoch 1 to 0.1935 in Epoch 5), indicating that the model is learning well on the clean data.
Avg Adv Loss:

This is the average loss on adversarial (perturbed) images. Lower values here suggest that the model is becoming more robust to adversarial attacks.
The sharp decline in Avg Adv Loss (from 0.7546 in Epoch 1 to 0.1588 in Epoch 5) indicates that the model is learning to correctly classify images even when small, intentional perturbations are added.
Avg Acc (Accuracy on clean data):

This is the average accuracy on clean images across the epoch. It improves from 83.61% in Epoch 1 to 92.83% by Epoch 5, showing that the model is generalizing better on clean images.
Avg Adv Acc (Accuracy on adversarial data):

This is the average accuracy on adversarial images, and it’s a key metric for evaluating robustness.
Avg Adv Acc starts lower (71.88%) because adversarial examples are challenging. However, it improves significantly to 94.85% by Epoch 5, indicating that the model is now robust against adversarial attacks, almost matching its accuracy on clean images.
Overall Interpretation
These results show that the model is learning both the original task of classifying clean images and gaining resilience to adversarial perturbations. By the final epoch, Avg Adv Acc is comparable to Avg Acc, meaning the model has learned to defend well against adversarial attacks, achieving high accuracy on both clean and perturbed images. This adversarial training approach has effectively enhanced the model’s robustness.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method

import numpy as np


In [2]:
# Load FashionMNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform),
    batch_size=1000, shuffle=False
)

In [3]:
# Define a ResNet50 model adapted for grayscale images and 10 classes
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.model = models.resnet50(weights=None)  # Use weights=None instead of pretrained=False
        # Modify the first conv layer to accept 1-channel input
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # Remove maxpool layer to reduce spatial downsampling
        self.model.maxpool = nn.Identity()
        # Modify the fully connected layer to output 10 classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.model(x)
        return x

# Normalize function
def normalize(tensor):
    return (tensor - 0.5) / 0.5

# Adversarial training with CleverHans for FGSM attack
def train(model, device, train_loader, optimizer, epsilon, epoch):
    model.train()
    total_loss, total_adv_loss = 0, 0
    correct, correct_adv = 0, 0
    total = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Standard forward pass
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        total_loss += loss.item()

        # Compute accuracy for clean data
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        # Generate adversarial examples using CleverHans FGSM without clipping
        adv_data = fast_gradient_method(
            model_fn=model,
            x=data,
            eps=epsilon,
            norm=np.inf,
            clip_min=None,  # Disable clipping to mimic the custom function
            clip_max=None,
            y=target,
            targeted=False
        )

        # If needed, normalize adv_data to match model's expected input
        adv_data = normalize(adv_data)

        # Get the model's prediction on the adversarial example
        with torch.no_grad():
            adv_output = model(normalize(adv_data))  # Normalize the adversarial example before prediction
            _, adv_pred_label = torch.max(adv_output, 1)
        
        adv_loss = F.cross_entropy(adv_output, target)
        total_adv_loss += adv_loss.item()

        # Compute accuracy for adversarial data
        adv_pred = adv_output.argmax(dim=1, keepdim=True)
        correct_adv += adv_pred.eq(target.view_as(adv_pred)).sum().item()

        # Combined loss
        total_batch_loss = (loss + adv_loss) / 2

        # Backward pass and optimization
        total_batch_loss.backward()
        optimizer.step()

        total += target.size(0)

        # Log information for each batch
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] '
                  f'Loss: {loss.item():.4f} | Adv Loss: {adv_loss.item():.4f} | '
                  f'Acc: {100. * correct / total:.2f}% | Adv Acc: {100. * correct_adv / total:.2f}%')

    # Log average loss and accuracy for epoch
    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    avg_acc = 100. * correct / total
    avg_adv_acc = 100. * correct_adv / total
    print(f'==> Epoch: {epoch+1} | Avg Loss: {avg_loss:.4f} | Avg Adv Loss: {avg_adv_loss:.4f} | '
          f'Avg Acc: {avg_acc:.2f}% | Avg Adv Acc: {avg_adv_acc:.2f}%')


In [4]:

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Using device: {device}")

# Training loop
epochs = 5
epsilon = 0.1  
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epsilon, epoch)
    print(f"Epoch {epoch+1}/{epochs} completed\n")

print("Adversarial training completed.")

Using device: cuda
Epoch: 1 [0/60000] Loss: 2.3952 | Adv Loss: 2.4109 | Acc: 18.75% | Adv Acc: 18.75%
Epoch: 1 [6400/60000] Loss: 0.5175 | Adv Loss: 1.1748 | Acc: 67.76% | Adv Acc: 49.60%
Epoch: 1 [12800/60000] Loss: 0.5394 | Adv Loss: 1.4170 | Acc: 74.30% | Adv Acc: 54.31%
Epoch: 1 [19200/60000] Loss: 0.4681 | Adv Loss: 1.7106 | Acc: 77.56% | Adv Acc: 55.34%
Epoch: 1 [25600/60000] Loss: 0.5933 | Adv Loss: 2.1269 | Acc: 79.73% | Adv Acc: 55.10%
Epoch: 1 [32000/60000] Loss: 0.3069 | Adv Loss: 1.4107 | Acc: 81.17% | Adv Acc: 55.05%
Epoch: 1 [38400/60000] Loss: 0.2571 | Adv Loss: 1.8583 | Acc: 82.22% | Adv Acc: 54.82%
Epoch: 1 [44800/60000] Loss: 0.4777 | Adv Loss: 2.1086 | Acc: 83.11% | Adv Acc: 54.69%
Epoch: 1 [51200/60000] Loss: 0.2876 | Adv Loss: 2.8580 | Acc: 83.89% | Adv Acc: 54.78%
Epoch: 1 [57600/60000] Loss: 0.3013 | Adv Loss: 2.1303 | Acc: 84.54% | Adv Acc: 54.70%
==> Epoch: 1 | Avg Loss: 0.4300 | Avg Adv Loss: 1.7482 | Avg Acc: 84.77% | Avg Adv Acc: 54.77%
Epoch 1/5 completed

